In [100]:
from chart_studio import plotly
import plotly.io as pio
import plotly.graph_objs as go
import pandas as pd
import aws_helper

In [25]:
redshift_endpoint = aws_helper.DB_HOST
redshift_user = aws_helper.DB_USER
redshift_pass = aws_helper.DB_PASSWORD
port = int(aws_helper.DB_PORT)
dbname = aws_helper.DB

In [26]:
import psycopg2
conn = psycopg2.connect(
    host=redshift_endpoint,
    user=redshift_user,
    port=port,
    password=redshift_pass,
    dbname=dbname)
cur = conn.cursor()

In [65]:
df = pd.read_sql_query("""
SELECT sp.location, a.name, COUNT(sp.artist_id) as artist_count FROM songplays sp JOIN artists a ON a.artist_id=sp.artist_id GROUP BY sp.location,a.name ORDER BY artist_count DESC;
""", conn)
df.head()

,location,name,artist_count
0,"Lansing-East Lansing, MI",Dwight Yoakam,95
1,"San Francisco-Oakland-Hayward, CA",Dwight Yoakam,76
2,"Lake Havasu City-Kingman, AZ",Dwight Yoakam,57
3,"Atlanta-Sandy Springs-Roswell, GA",Dwight Yoakam,57
4,"Chicago-Naperville-Elgin, IL-IN-WI",Dwight Yoakam,57


In [66]:
locations = df['location']
df[['city','state']] = df.location.str.split(',', expand=True)
df = df.sort_values(by=['state'])
df.head()

,location,name,artist_count,city,state
43,"Birmingham-Hoover, AL",Muse,16,Birmingham-Hoover,AL
23,"Birmingham-Hoover, AL",Dwight Yoakam,19,Birmingham-Hoover,AL
287,"Birmingham-Hoover, AL",Celtic Woman,2,Birmingham-Hoover,AL
402,"Birmingham-Hoover, AL",Fukkk Offf,1,Birmingham-Hoover,AL
214,"Birmingham-Hoover, AL",James Taylor,3,Birmingham-Hoover,AL


In [67]:
# Adapted from https://plotly.com/python/v3/amazon-redshift/

fig = go.Figure(data=go.Heatmap(
        z=df['artist_count'],
        x=df['state'],
        y=df['name'],
        colorscale='Portland'))

fig.update_layout(
    title='Artist Popularity By State',
    xaxis_nticks=35, yaxis_nticks=500, height=3600)

fig.show()

In [68]:
# Remove artist counts above 15, the mean count to get a more fine grained view of smaller counts

df_reduced = df[df.artist_count <= 15]
fig = go.Figure(data=go.Heatmap(
        z=df_reduced['artist_count'],
        x=df_reduced['state'],
        y=df_reduced['name'],
        colorscale='Portland'))

fig.update_layout(
    title='Artist Popularity By State, Counts 15 or below',
    xaxis_nticks=35, yaxis_nticks=500, height=3600)

fig.show()

In [74]:
df_artist_loc = pd.read_sql_query("""
SELECT latitude, longitude, name FROM artists;
""", conn)
df_artist_loc.dropna(subset = ['latitude', 'longitude'], inplace=True)
df_artist_loc.head()

,latitude,longitude,name
0,27.49550,-82.57807,We The Kings
1,51.43558,-2.57518,Tricky
2,42.18419,-71.71818,The Dear Hunter
4,53.40977,-2.97848,Atomic Kitten
5,34.05349,-118.24532,Katy Perry


In [101]:
# Adapted from https://plotly.com/python/ipython-notebook-tutorial/

artist_lat = df_artist_loc.latitude
artist_lon = df_artist_loc.longitude
artist_name = df_artist_loc.name

data = [
    go.Scattermapbox(
        lat=artist_lat,
        lon=artist_lon,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=artist_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=artist_lat,
        lon=artist_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        text=artist_name,
        hoverinfo='text'
    )]


layout = go.Layout(
    title='Artist Locations',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=#<Mapbox token>,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
pio.write_html(fig, file='jupyter-artist-location.html', auto_open=True)

In [93]:
# User level by location

df = pd.read_sql_query("""
SELECT sp.location, u.level, COUNT(u.level) as level_count FROM songplays sp JOIN users u ON u.user_id=sp.user_id GROUP BY sp.location, u.level ORDER BY level_count DESC;
""", conn)
locations = df['location']
df[['city','state']] = df.location.str.split(',', expand=True)
df = df.sort_values(by=['state'])
df.head()

,location,level,level_count,city,state
11,"Birmingham-Hoover, AL",paid,31,Birmingham-Hoover,AL
3,"Lake Havasu City-Kingman, AZ",paid,77,Lake Havasu City-Kingman,AZ
33,"Phoenix-Mesa-Scottsdale, AZ",free,3,Phoenix-Mesa-Scottsdale,AZ
1,"San Francisco-Oakland-Hayward, CA",paid,118,San Francisco-Oakland-Hayward,CA
30,"Salinas, CA",free,4,Salinas,CA


In [97]:
fig = go.Figure(data=go.Heatmap(
        z=df['level_count'],
        x=df['state'],
        y=df['level'],
        colorscale='Portland'))

fig.update_layout(
    title='Songplays by level per state',
    xaxis_nticks=35)

fig.show()